In [17]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature 
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features

    def fit(self, x, y):
        self.root = self._grow_tree(x, y)

    def _grow_tree(self, X, y, depth=0):
        if depth > self.max_depth or len(np.unique(y)) == 1:
            return Node(value=y[0])

        feature, threshold = self._best_split(X, y, self.n_features)
        
        left, right = self._split(X[:, feature], threshold)

        left_subtree = self._grow_tree(X[left], y[left], depth + 1)
        right_subtree = self._grow_tree(X[right], y[right], depth + 1)

        return Node(feature=feature, threshold=threshold, left=left_subtree, right=right_subtree)
        

    def _best_split(self, X, y, feat_idxs):
        best_feature = 0
        best_threshold = 0
        best_info_gain = 0
        en_parent = self._entropy(y)
        
        for feature in range(feat_idxs):
            unique_values = np.unique(X[:, feature])

            for i in range(len(unique_values)-1):
                threshold = (unique_values[i]+unique_values[i+1])/2

                left = np.where(X[:, feature] <= threshold)[0]
                right = np.where(X[:, feature] > threshold)[0]

                en_left = self._entropy(y[left])
                en_right = self._entropy(y[right])
                left_n = len(y[left])
                right_n = len(y[right])

                info_gain = en_parent - ((left_n) / (left_n + right_n) * en_left) - (right_n / (left_n + right_n) * en_right)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold
                

    def _split(self, X_column, split_thresh):
        left = X_column[:] <= split_thresh
        right = X_column[:] > split_thresh

        return left, right

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist/len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])


In [18]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

data = datasets.load_iris()
x, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

clf = DecisionTree(max_depth=10, n_features=len(data.data[0]))
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, predictions)
print(acc)

1.0
